In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

In [9]:
spark = SparkSession.builder.appName("get uf").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 08:19:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/08 08:19:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
date = datetime.strftime(datetime(2022,1,1,12,32,20), '%Y-%m-%d %H-%M-%S')
date

'2022-01-01 12-32-20'

In [51]:
df = spark.read.parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/teste')

In [21]:
df.show()

+-----------------+-----------+------------------+----+-----+--------------------+
|qt_votos_nominais|total_votos|       porcentagem|year|sg_uf|        nm_candidato|
+-----------------+-----------+------------------+----+-----+--------------------+
|          4156217|    9560111| 43.47456844381828|2022|   RJ|LUIZ INACIO LULA ...|
|          4156217|    9560111| 43.47456844381828|2022|   RJ|LUIZ INACIO LULA ...|
|           802502|    2404455| 33.37562982047907|2022|   PB|JAIR MESSIAS BOLS...|
|           802502|    2404455| 33.37562982047907|2022|   PB|JAIR MESSIAS BOLS...|
|           467065|    2018448| 23.13980840725151|2022|   PI|JAIR MESSIAS BOLS...|
|           467065|    2018448| 23.13980840725151|2022|   PI|JAIR MESSIAS BOLS...|
|           200547|     390465| 51.36106949406477|2022|   AP|JAIR MESSIAS BOLS...|
|           200547|     390465| 51.36106949406477|2022|   AP|JAIR MESSIAS BOLS...|
|          3640933|    5439765| 66.93180679680097|2022|   PE|LUIZ INACIO LULA ...|
|   

In [52]:
df = df.withColumn('nm_candidato', 
                when(df.nm_candidato == 'JAIR MESSIAS BOLSONARO',regexp_replace('nm_candidato','JAIR MESSIAS BOLSONARO','Bolsonaro')) \
                .when(df.nm_candidato == 'LUIZ INACIO LULA DA SILVA',regexp_replace('nm_candidato','LUIZ INACIO LULA DA SILVA','Lula'))    
                )

In [53]:
df = df.drop_duplicates()

In [54]:
from pyspark.sql.window import Window

In [114]:
df.where(df.sg_uf == 'RJ').show()

+-----------------+-----------+-----------------+----+-----+------------+
|qt_votos_nominais|total_votos|      porcentagem|year|sg_uf|nm_candidato|
+-----------------+-----------+-----------------+----+-----+------------+
|          5403894|    9560111|56.52543155618172|2022|   RJ|   Bolsonaro|
+-----------------+-----------+-----------------+----+-----+------------+



In [55]:
percent_uf = Window.partitionBy("sg_uf").orderBy(col("porcentagem").desc())

In [58]:
df = df.withColumn("row",row_number().over(percent_uf)) \
  .filter(col("row") == 1).drop("row")

In [116]:
df.show()

+-----------------+-----------+------------------+----+-----+------------+
|qt_votos_nominais|total_votos|       porcentagem|year|sg_uf|nm_candidato|
+-----------------+-----------+------------------+----+-----+------------+
|           287750|     409316| 70.30020815213673|2022|   AC|   Bolsonaro|
|           976831|    1664658| 58.68058183723023|2022|   AL|        Lula|
|          1004991|    1966732| 51.09953974410342|2022|   AM|        Lula|
|           200547|     390465| 51.36106949406477|2022|   AP|   Bolsonaro|
|          6097815|    8454843| 72.12215531382428|2022|   BA|        Lula|
|          3807891|    5442368| 69.96753986499994|2022|   CE|        Lula|
|          1041331|    1770626|58.811459901752265|2022|   DF|   Bolsonaro|
|          1282145|    2208912| 58.04418645921612|2022|   ES|   Bolsonaro|
|          2193041|    3735156|58.713504871014756|2022|   GO|   Bolsonaro|
|          2668425|    3751174|  71.1357297741987|2022|   MA|        Lula|
|          6190960|   123

In [135]:
sent = spark.read.parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/ufs')

In [136]:
sent.show()

+--------------------+-------------------+-------------------+--------------------+----------+---------+--------------------+------+-----+---+----+---------+---------+
|                 geo|                 id|         tweet_date|          tweet_text|sentimento|pontuacao|              cidade|  pais|month|day|year|       uf|candidato|
+--------------------+-------------------+-------------------+--------------------+----------+---------+--------------------+------+-----+---+----+---------+---------+
|   Sao Paulo, Brazil|1597943225108418560|2022-11-30 13:18:47|@belloniviviane @...|  negativo|  -0.6808|           Sao Paulo|Brazil|   11| 30|2022|São Paulo|Bolsonaro|
|    Sorocaba, Brasil|1597943534434549761|2022-11-30 13:20:01|@kimpaim bela fot...|  positivo|    0.296|            Sorocaba|Brasil|   11| 30|2022|São Paulo|Bolsonaro|
|São José do Rio P...|1597943708707852289|2022-11-30 13:20:43|@luizpatriota39 a...|  negativo|   -0.296|São José do Rio P...|Brasil|   11| 30|2022|São Paulo|Bol

In [137]:
teste = sent.groupBy(['candidato','uf','sentimento']).agg(count('sentimento')).withColumnRenamed("count(sentimento)", "total_sentimento")

In [138]:
temp = teste.groupBy(['candidato','uf']).agg(sum('total_sentimento')).withColumnRenamed('sum(total_sentimento)','total_uf')
temp = temp.withColumnRenamed("uf", "uf_drop").withColumnRenamed('candidato','candidato_drop')

In [139]:
temp.show()

+--------------+------------------+--------+
|candidato_drop|           uf_drop|total_uf|
+--------------+------------------+--------+
|          Lula|  Distrito Federal|      60|
|     Bolsonaro|              erro|      23|
|     Bolsonaro|        Pernambuco|      58|
|          Lula|           Alagoas|      12|
|     Bolsonaro|             Piauí|      29|
|     Bolsonaro|    Espírito Santo|      31|
|     Bolsonaro|             Ceará|      64|
|          Lula|             Goiás|      38|
|     Bolsonaro|           Sergipe|      15|
|          Lula|             Bahia|      32|
|     Bolsonaro|         Tocantins|      12|
|          Lula|          Maranhão|      11|
|     Bolsonaro|  Distrito Federal|      86|
|     Bolsonaro|Mato Grosso do Sul|      44|
|     Bolsonaro|           Roraima|       3|
|          Lula|    Santa Catarina|      59|
|     Bolsonaro|         São Paulo|     630|
|     Bolsonaro|          Amazonas|      41|
|     Bolsonaro|             Bahia|      45|
|         

In [150]:
final = teste.join(temp, [teste.uf == temp.uf_drop, teste.candidato == temp.candidato_drop],'left').drop('uf_drop','candidato_drop')

In [141]:
final.printSchema()

root
 |-- candidato: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- sentimento: string (nullable = true)
 |-- total_sentimento: long (nullable = false)
 |-- total_uf: long (nullable = true)



In [142]:
final.where(final.uf == 'Distrito Federal').show()

+---------+----------------+----------+----------------+--------+
|candidato|              uf|sentimento|total_sentimento|total_uf|
+---------+----------------+----------+----------------+--------+
|Bolsonaro|Distrito Federal|  positivo|              14|      86|
|Bolsonaro|Distrito Federal|    neutro|              46|      86|
|Bolsonaro|Distrito Federal|  negativo|              26|      86|
|     Lula|Distrito Federal|  negativo|              26|      60|
|     Lula|Distrito Federal|  positivo|               6|      60|
|     Lula|Distrito Federal|    neutro|              28|      60|
+---------+----------------+----------+----------------+--------+



In [151]:
final = final.withColumn('porcentagem', (col('total_sentimento') * 100)/col('total_uf'))

In [152]:
final.show()

+---------+-----------------+----------+----------------+--------+------------------+
|candidato|               uf|sentimento|total_sentimento|total_uf|       porcentagem|
+---------+-----------------+----------+----------------+--------+------------------+
|Bolsonaro|   Rio de Janeiro|    neutro|             203|     417|  48.6810551558753|
|Bolsonaro|           Paraná|  negativo|              22|      95|23.157894736842106|
|Bolsonaro|   Rio de Janeiro|  positivo|              72|     417| 17.26618705035971|
|Bolsonaro|     Minas Gerais|  positivo|              32|     177| 18.07909604519774|
|Bolsonaro|           Paraná|  positivo|              27|      95| 28.42105263157895|
|     Lula|     Minas Gerais|    neutro|              52|     107|48.598130841121495|
|     Lula|        São Paulo|  negativo|             112|     370| 30.27027027027027|
|Bolsonaro|           Paraná|    neutro|              46|      95|48.421052631578945|
|     Lula|     Minas Gerais|  positivo|              

In [153]:
percent_uf = Window.partitionBy("uf").orderBy(col("porcentagem").desc())

In [154]:
final = final.where(final.sentimento == 'negativo')

In [155]:
final = final.withColumn("row",row_number().over(percent_uf)) \
  .filter(col("row") == 1).drop("row")

In [156]:
final.show()

+---------+-------------------+----------+----------------+--------+------------------+
|candidato|                 uf|sentimento|total_sentimento|total_uf|       porcentagem|
+---------+-------------------+----------+----------------+--------+------------------+
|     Lula|               Acre|  negativo|               1|       4|              25.0|
|Bolsonaro|            Alagoas|  negativo|               5|      18| 27.77777777777778|
|Bolsonaro|              Amapá|  negativo|               2|       8|              25.0|
|Bolsonaro|           Amazonas|  negativo|              12|      41| 29.26829268292683|
|     Lula|              Bahia|  negativo|              10|      32|             31.25|
|Bolsonaro|              Ceará|  negativo|              21|      64|           32.8125|
|     Lula|   Distrito Federal|  negativo|              26|      60|43.333333333333336|
|     Lula|     Espírito Santo|  negativo|              10|      29| 34.48275862068966|
|Bolsonaro|              Goiás| 